In [1]:
import os
import sys
import json # print(json.dumps(text, indent=2))
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

sparql = SPARQLWrapper("http://sparql.archives-ouvertes.fr/sparql")
sparql.setReturnFormat(JSON)
sparql.setTimeout(10000000)

pd.set_option('display.max_colwidth', None)

def execute(query):
    queryString = """
        prefix hal:	<http://data.archives-ouvertes.fr/schema/>
        prefix vcard2006: <http://www.w3.org/2006/vcard/ns#>
        prefix ns8:	<http://fr.dbpedia.org/resource/>
        prefix org: <http://www.w3.org/ns/org#>
        prefix skos: <http://www.w3.org/2004/02/skos/core#>
        prefix dcterms: <http://purl.org/dc/terms/>
        prefix fabio: <http://purl.org/spar/fabio/>
        prefix ore:	<http://www.openarchives.org/ore/terms/>
        %s
        """ % query
    sparql.setQuery(queryString)
    response = sparql.query().convert()
    df = pd.json_normalize(response['results']['bindings'])
    filter_col = [col for col in df if not col.endswith(('.type', '.datatype'))]
    return df[filter_col]

# 42970 authors with aggregate: 42808 with name, 162 without name
# 1517974 authors without aggregate: 1346390 with name, 171584 without name

### La requête permettant de récupérer la liste des auteurs qui ont changé de pays plus qu'une fois

In [2]:
query_authors = """
    select distinct ?author count(distinct ?country)
    where {{
        ?doc dcterms:creator ?n.
        ?n hal:person ?author; hal:structure ?lab.
        ?lab vcard2006:country-name ?country.
    }}
    group by ?author
    having (count(distinct ?country) > 2)
    limit {0} offset {1}
"""

authors = execute(query_authors.format(100, 0))
# authors.head(10)

### La requête permettant de récupérer les informations nécessaires pour un auteur en particulier

In [3]:
query_data = """
    select distinct
        ?author 
        ?doc xsd:date(?issued) as ?issuedAt 
        ?lab ?labName 
        replace(str(?country), 'http://fr.dbpedia.org/resource/', '') AS ?country 
        concat(str(xsd:date(?available)), '') as ?availableAt
        concat(str(?nameAgg), '') as ?nameAgg 
        concat(str(?nameDoc), '') as ?nameDoc
        concat(str(?adress), '') as ?adress
    where {{
        ?doc dcterms:creator ?n;
            dcterms:issued ?issued.
            optional {{ ?doc dcterms:available ?available }}
        ?n hal:person {0}; hal:structure ?lab.
        ?lab skos:prefLabel ?labName;
            vcard2006:country-name ?country.
            optional {{ ?lab org:siteAddress ?adress }}
        optional {{ ?authorAgg ore:aggregates {0} optional {{ ?authorAgg foaf:name ?nameAgg }} }}
        optional {{ {0} foaf:name ?nameDoc }}
        bind( if(bound(?authorAgg), ?authorAgg, {0} ) as ?author )
        filter(bound(?nameAgg) or bound(?nameDoc))
    }}
    limit {1} offset {2}
"""
limit = 10000

### Exporter les données des auteurs sélectionnés dans un fichier CSV

In [4]:
mode = 'w'
for row in authors.values.tolist():
    authorId = '<'+ row[0] +'>'

    offset = 0
    result = execute( query_data.format( authorId, limit, offset) ).fillna('')
    while len(result):
        header = False if mode == 'a' else list(result.columns)
        result.sort_values(['author.value', 'issuedAt.value'], axis = 0, ascending = True,
                           inplace = True, na_position ='last')
        result.to_csv('output/data.csv', sep=',', index=False, header=header, mode=mode)
        mode = 'a'
        offset += limit
        result = execute( query_data.format( authorId, limit, offset) ) # .fillna('0')

### Lire le fichier CSV exporté précédemment

In [5]:
data = pd.read_csv("output/data.csv").fillna('')
# list(data.columns)

### Regrouper les données par auteur et par année et exporter le résultat dans un fichier JSON

In [6]:
byAuthorDict = {}
for row in data.values.tolist():
    authorURI = row[0]
    year, mont, date = row[2].split('-')

    if authorURI not in byAuthorDict:
        byAuthorDict[ authorURI ] = {}
    if year not in byAuthorDict[ authorURI ]:
        byAuthorDict[ authorURI ][ year ] = []

    byAuthorDict[ authorURI ][ year ].append({
        "name1": row[7],
        "name2": row[8],
        "docURI": row[1],
        # "labURI": row[3],
        "labName": row[4],
        "country": row[5],
        "adress": row[9],
        "issuedAt": row[2],
        "availableAt": row[6],
    })

byAuthorList = []
for authorURI in byAuthorDict:
    names = set()
    authorData = list()
    for year in byAuthorDict[ authorURI ]:
        docs = list()
        for row in byAuthorDict[ authorURI ][ year ]:
            if row['name1']:
                names.add(row['name1'])
            if row['name2']:
                names.add(row['name2'])
            docs.append( {k:row[k] for k in ["docURI", 
                                             "labName", "country", "adress"]} )
        authorData.append({
            "year": year,
            "docs": docs
        })

    byAuthorList.append({
        "authorURI": authorURI, # .replace("https://data.archives-ouvertes.fr/author/", ""),
        "name": list(names)[0] if len(names) == 1 else list(names),
        "data": authorData
    })
with open('output/finaldata.json', 'w', encoding='utf-8') as f:
    json.dump(byAuthorList, f, ensure_ascii=False, indent=2)